# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for  Google and ollama, as well as OpenAI.

## Setting up your keys

If you haven't done so already, you could now create API keys for Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Google! You can see me do it, and focus on OpenAI for the course. You could also substitute and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  

For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [ ]:
# imports

import os
from dotenv import load_dotenv
import google.generativeai
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI and Google
# Having problems with API files? You can use openai = OpenAI(api_key="your-key-here") and same for claude
# Having problems with Google Gemini setup? Then just skip Gemini; you'll get all the experience you need from GPT and Claude.

openai = OpenAI()

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [ ]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [ ]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [ ]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4o-mini and gemini-1.5-flash
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
gemini_model = "gemini-1.5-flash"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

gemini_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_instance = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=gemini_system
)

gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

In [ ]:
def construct_joined_user_msg(msg1):
    return msg1 

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content":construct_joined_user_msg(gemini)})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_gemini(return_msgs=False):
    messages = []
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "user", "parts": construct_joined_user_msg(gpt)})
        messages.append({"role": "model", "parts": gemini})
    messages.append({"role": "user", "parts": construct_joined_user_msg(gpt_messages[-1])})
    if return_msgs: return messages
    message = gemini_instance.generate_content(messages)
    return message.text

In [ ]:
# def call_gemini():
#     messages = []
#     for gpt, gemini_message in zip(gpt_messages, gemini_messages):
#         messages.append({"role": "user", "content": gpt})
#         messages.append({"role": "assistant", "content": gemini_message})
#     messages.append({"role": "user", "content": gpt_messages[-1]})
#     message = google.generativeai.GenerativeModel(gemini_model).generate_content(
#         # model=gemini_model,
#         # system=gemini_system,
#         # messages=messages,
#         # max_tokens=500
#     )
#     # model = google.generativeai.GenerativeModel("gemini-1.5-flash")
#     # messages = model.generate_content(messages)
    
#     # return message.text

In [ ]:
call_gemini()

In [ ]:
call_gpt()

In [ ]:
print(f"GPT:\n{gpt_messages[0]}\n")
# print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")


for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT :\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    

    gemini_next = call_gemini()
    print(f"Gemini :\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

# See the community-contributions folder

For a great variation with a 3-way bringing Gemini into the conversation!

Try doing this yourself before you look in the folder.

## Additional exercise

Try adding in an Ollama model in to the conversation.

## Business relevance

This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.